In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv('exoplanet_data.csv')
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
#target value counts
df['koi_disposition'].value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [4]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact','koi_duration', 'koi_depth', 'koi_prad', 
       'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff',  'koi_slogg', 
       'koi_srad',  'ra', 'dec','koi_kepmag']]

In [5]:
selected_features.shape

(6991, 20)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
target = df['koi_disposition']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(selected_features,target,test_size = 0.2)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6351,0,0,0,0,1.320499,132.61488,0.150,1.184,204.0,18.94,4477,95168.65,12.0,1,5172,2.693,13.454,282.93994,49.050289,11.466
3369,0,0,0,0,5.759022,132.16301,0.329,2.687,70.8,0.91,1112,362.12,9.9,1,6247,4.433,1.066,295.57159,49.607479,14.199
6229,1,0,0,0,45.680876,142.35740,0.206,2.293,350.3,14.50,1215,514.91,3.7,1,4835,2.608,7.917,293.66245,44.527512,13.409
5518,0,1,1,1,40.879068,163.40560,0.988,6.562,499.2,3.35,533,19.12,12.2,1,6072,4.510,0.923,292.34671,42.402939,15.823
2157,0,0,0,0,0.896382,132.29651,0.332,1.113,136.4,1.00,1810,2524.81,11.7,1,5837,4.502,0.843,298.53967,40.183659,14.603


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(selected_features)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Label-endoce data set
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [12]:
# one-hot encoding
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [13]:
y_train_categorical.shape

(5592, 3)

# Train the Model
## Deep learning

In [14]:
# define the model layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
# 20 selected features and 3 output classes
model.add(Dense(units=100,activation='relu',input_dim=20))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=3,activation='softmax'))

In [15]:
# Compile the model
model.compile(optmizer = 'adam',loss='categorical_crossentropy',metrics= ['accuracy'])

In [16]:
# Fit the model
model.fit(X_train_scaled,y_train_categorical,epochs = 150,shuffle=True,verbose=2)

Train on 5592 samples
Epoch 1/150
5592/5592 - 1s - loss: 0.4980 - accuracy: 0.7412
Epoch 2/150
5592/5592 - 0s - loss: 0.3790 - accuracy: 0.7954
Epoch 3/150
5592/5592 - 0s - loss: 0.3718 - accuracy: 0.7979
Epoch 4/150
5592/5592 - 0s - loss: 0.3682 - accuracy: 0.8035
Epoch 5/150
5592/5592 - 0s - loss: 0.3634 - accuracy: 0.8058
Epoch 6/150
5592/5592 - 0s - loss: 0.3632 - accuracy: 0.8040
Epoch 7/150
5592/5592 - 0s - loss: 0.3603 - accuracy: 0.8069
Epoch 8/150
5592/5592 - 0s - loss: 0.3595 - accuracy: 0.8058
Epoch 9/150
5592/5592 - 0s - loss: 0.3578 - accuracy: 0.8097
Epoch 10/150
5592/5592 - 0s - loss: 0.3556 - accuracy: 0.8147
Epoch 11/150
5592/5592 - 0s - loss: 0.3534 - accuracy: 0.8171
Epoch 12/150
5592/5592 - 0s - loss: 0.3528 - accuracy: 0.8214
Epoch 13/150
5592/5592 - 0s - loss: 0.3484 - accuracy: 0.8199
Epoch 14/150
5592/5592 - 0s - loss: 0.3495 - accuracy: 0.8149
Epoch 15/150
5592/5592 - 0s - loss: 0.3467 - accuracy: 0.8253
Epoch 16/150
5592/5592 - 0s - loss: 0.3475 - accuracy: 0.

Epoch 133/150
5592/5592 - 0s - loss: 0.2371 - accuracy: 0.8952
Epoch 134/150
5592/5592 - 0s - loss: 0.2402 - accuracy: 0.8954
Epoch 135/150
5592/5592 - 0s - loss: 0.2377 - accuracy: 0.8995
Epoch 136/150
5592/5592 - 0s - loss: 0.2338 - accuracy: 0.9008
Epoch 137/150
5592/5592 - 0s - loss: 0.2354 - accuracy: 0.9033
Epoch 138/150
5592/5592 - 0s - loss: 0.2344 - accuracy: 0.9000
Epoch 139/150
5592/5592 - 0s - loss: 0.2335 - accuracy: 0.8990
Epoch 140/150
5592/5592 - 0s - loss: 0.2358 - accuracy: 0.9009
Epoch 141/150
5592/5592 - 0s - loss: 0.2360 - accuracy: 0.9018
Epoch 142/150
5592/5592 - 0s - loss: 0.2346 - accuracy: 0.9011
Epoch 143/150
5592/5592 - 0s - loss: 0.2320 - accuracy: 0.9040
Epoch 144/150
5592/5592 - 0s - loss: 0.2297 - accuracy: 0.9056
Epoch 145/150
5592/5592 - 0s - loss: 0.2349 - accuracy: 0.9031
Epoch 146/150
5592/5592 - 0s - loss: 0.2299 - accuracy: 0.9061
Epoch 147/150
5592/5592 - 0s - loss: 0.2262 - accuracy: 0.9016
Epoch 148/150
5592/5592 - 0s - loss: 0.2317 - accuracy:

In [17]:
# quantifying the model
# evaluate the model using the testing data
model_loss,model_accuracy = model.evaluate(X_test_scaled,y_test_categorical,verbose=2)
print(f'Loss:{model_loss},Accuracy:{model_accuracy}')

1399/1 - 0s - loss: 0.2931 - accuracy: 0.8721
Loss:0.2946821451187134,Accuracy:0.872051477432251


In [18]:
# Use the first 15 test data values to make a prediction and compare with true value
raw_predictions = model.predict_classes(X_test_scaled[:10])
predictions = list(label_encoder.inverse_transform(raw_predictions))
pd.DataFrame({'Predicted class':predictions,'Actual labels':list(y_test[:10])},columns = ['Predicted class','Actual labels'])

,Predicted class,Actual labels
0,CANDIDATE,CONFIRMED
1,CONFIRMED,CONFIRMED
2,FALSE POSITIVE,FALSE POSITIVE
3,FALSE POSITIVE,FALSE POSITIVE
4,CONFIRMED,CONFIRMED
5,CONFIRMED,CONFIRMED
6,FALSE POSITIVE,FALSE POSITIVE
7,CANDIDATE,CANDIDATE
8,CANDIDATE,CANDIDATE
9,FALSE POSITIVE,FALSE POSITIVE


In [19]:
# Save the model
model.save("model_DeepLearning.h5")